<a href="https://colab.research.google.com/github/jenskuehne-cmd/Colab_Skripte_OEE/blob/main/Notebooks/RoleMappingAspire/KAU_RM_Daten_manipulieren.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Notebook Beschreibung

Dieses Notebook dient der Analyse und Bereinigung von Benutzerdaten, die aus einer Google Sheet-Tabelle namens "RM_KAU" geladen werden. Die Hauptziele sind:

1.  **Daten laden**: Es stellt eine Verbindung zu Google Sheets her und lädt die Daten aus dem Tabellenreiter "RM_KAU" in einen Pandas DataFrame.
2.  **Filterung nach Rollen**: Es identifiziert und filtert Benutzer, die eine spezifische "Craftsman"-Rolle innehaben (genauer gesagt: "CH-JE-0800_1090_CRAFTMAN").
3.  **Identifizierung unzugeordneter "Craftsman"**: Es filtert "Craftsman"-Benutzer heraus, bei denen die Zuweisung noch nicht abgeschlossen ist (`Mapping erledigt` ist nicht 'TRUE') und die noch keine spezifischen "Maintenance Technician"-Rollen (wie '1201_Maintenance Technician_KAU' oder '1201_Maintenance Technician_BSL') haben. Dies hilft, offene Zuweisungsaufgaben zu erkennen.
4.  **Analyse und Zusammenfassung**: Es erstellt Auswertungen, die die Anzahl der betroffenen Personen nach 'User - Organization' und 'User - Workday Position Title' gruppieren und quantifizieren. Dies gibt einen Überblick über die Verteilung der Rollen innerhalb der Organisation.
5.  **Duplikatsprüfung**: Es sucht nach Duplikaten in bestimmten Spalten (AB bis AH) der Tabelle "RM_KAU" und markiert diese visuell im Google Sheet (obwohl die aktuelle Implementierung die Markierung nicht dauerhaft speichert, wird das Problem erkannt).
6.  **Datenbereinigung**: Es korrigiert fehlerhafte Einträge in ausgewählten Spalten ("Maintenance KAU Roles", "Analyse Reporting", "Ersatzteilmanagem.", "Kalibrierung", "EWM KAU", "MDG EAM", "MDG MM"), die fälschlicherweise mit einem Komma beginnen, indem es dieses Komma direkt im Google Sheet entfernt.

Das Notebook hilft dabei, den Status der Rollenzuweisung zu überwachen, potenzielle Probleme (wie Duplikate oder fehlerhafte Einträge) zu identifizieren und die notwendigen Korrekturen vorzunehmen.
```

##KAU Tabelle einlesen

*die anderen müssen noch hinzugefügt werden*

In [ ]:
%%capture
# Installieren Sie die notwendigen Bibliotheken
!pip install --upgrade gspread pandas

# Importieren der benötigten Bibliotheken
import gspread
import pandas as pd
from google.colab import auth

# Authentifizierung für Google Drive und Google Sheets
auth.authenticate_user()

# Autorisieren des Zugriffs
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet öffnen (URL oder Sheet-ID einfügen)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)

# Tabellenreiter "RM_KAU" auswählen
worksheet = sheet.worksheet('RM_KAU')

# Daten als Pandas DataFrame laden
data = pd.DataFrame(worksheet.get_all_records())

# Die ersten 3 Zeilen der Tabelle anzeigen
print(data.head(3))


###Rollen mit Craftsman bisher zugeteilt
Gibt die Möglichkeit danach nach *User - Workday Position Title* zu filtern um ggf. automatisch den Compliance Menschen die ROllen zuzuteilen

Todo:
- es gibt compliance Menschen in der Produktion und auch die Prod. PM/PKs müssen angeschaut werden. Viele von denen schreiben Orders zB für die RV


Nachfolgender Code muss noch angepassst werden,
- dass nur **nicht** erledigte angezeigt werden
- das nicht angezeigt wird wenn bereits in Spalte *'Maintenance KAU Roles'* ein
 - *1201_Maintenance Technician_KAU* oder
 - *1201_Maintenance Technician_BSL* verteilt ist


##Alle USer mit P30 Craftsman Rolle

In [ ]:
# prompt: filtere mir alle UserID, Name, Vorname, Manager Name, User - Organization bei denen irgendwo in der Zelle in Spalte "P30 alle" folgendes String vorkommt: "CH-JE-0800_1090_CRAFTMAN" und schreibe in einen neuen Dataframe. sortiere nach User - Organization

# Filtere die Daten
filtered_data = data[data.apply(lambda row: row.astype(str).str.contains('CH-JE-0800_1090_CRAFTMAN').any(), axis=1)]

# Wähle die gewünschten Spalten aus
filtered_data = filtered_data[['User - Organization', 'User - Workday Position Title', 'UserID', 'Name', 'Vorname', 'Manager Name', 'P30 Rollen Asset', 'P30 alle']]
# Ergänzt User - Workday Position Title
# Sortiere nach 'User - Organization'
filtered_data = filtered_data.sort_values(by='User - Organization')

# Zeige die gefilterten Daten an
filtered_data

##MA mit Craftsman die noch nicht gemappt sind aus Abteilungen

###Alle MA namentlich ohne mapping mit Craftsman

In [ ]:
# @title
# prompt: ergänze den code, dass zusätzlich ein dataframe erzeugt wird mit einer Auswertung nach     'User - Organization' und
#     'User - Workday Position Title' wieviele Personen aus diesem Berich betroffen sind. eine quantitative angabe reicht.

import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Drive und Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet öffnen
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)
worksheet = sheet.worksheet('RM_KAU')
data = pd.DataFrame(worksheet.get_all_records())

# Filtere die Daten
filtered_data = data[data.apply(lambda row: row.astype(str).str.contains('CH-JE-0800_1090_CRAFTMAN').any(), axis=1)]

filtered_data = filtered_data[
    ~data['Mapping erledigt'].astype(str).str.contains('TRUE', na=False)
]

filtered_data = filtered_data[
    ~filtered_data['Maintenance KAU Roles'].astype(str).str.contains(
        '1201_Maintenance Technician_KAU', na=False
    )
]

filtered_data = filtered_data[
    ~filtered_data['Maintenance KAU Roles'].astype(str).str.contains(
        '1201_Maintenance Technician_BSL', na=False
    )
]

# Wähle die gewünschten Spalten aus
filtered_data = filtered_data[[
    'User - Organization',
    'User - Workday Position Title',
    'UserID',
    'Name',
    'Vorname',
    'Manager Name',
    'P30 Rollen Asset',
    'P30 alle',
]]

# Sortiere nach 'User - Organization'
filtered_data = filtered_data.sort_values(by='User - Organization')

# Zeige die gefilterten Daten an
filtered_data

###Sortiert nach Abt. und User Rolle Workday

In [ ]:
# @title
# Erstelle den neuen DataFrame mit der Auswertung
summaryAbtUser_df = filtered_data.groupby(['User - Organization', 'User - Workday Position Title'])['UserID'].count().reset_index(name='Anzahl Personen')
summaryAbtUser_df = summaryAbtUser_df.sort_values(by=['Anzahl Personen'], ascending=False)
print("\nZusammenfassung nach Organisation und Position:")
summaryAbtUser_df

###Anzahl nach Abteilung aufgelistet

In [ ]:
# @title
summaryAbt_df = filtered_data.groupby(['User - Organization'])['UserID'].count().reset_index(name='Anzahl Personen')
summaryAbt_df = summaryAbt_df.sort_values(by=['Anzahl Personen'], ascending=False)  # Add this line
print("\nZusammenfassung nach Organisation und Position (sortiert nach Anzahl Personen):")
summaryAbt_df



---



##Auswertung erledigte in KAU RM

In [ ]:
# @title
# prompt: mache eine auswertung wieviele Werte in der Spalte bei "Mapping erledigt" auf "TRUE" gesetzt sind im Verhältnis zu der Gesamtzahl Zeilen

# Zählen der Zeilen, bei denen "Mapping erledigt" auf "TRUE" steht
true_count = data[data['Mapping erledigt'] == 'TRUE'].shape[0]

# Gesamtzahl der Zeilen
total_rows = data.shape[0]

# Verhältnis berechnen
percentage = (true_count / total_rows) * 100

# Ausgabe der Ergebnisse
print(f"Anzahl der Zeilen mit 'Mapping erledigt' = TRUE: {true_count}")
print(f"Gesamtzahl der Zeilen: {total_rows}")
print(f"Anteil von 'Mapping erledigt' = TRUE an der Gesamtzahl der Zeilen: {percentage:.2f}%")

### Duplikate in KAU Liste checken
In den Spalten AB - AH , Zellen mit Duplikaten werden rot eingefärbt

In [ ]:
# Installieren Sie die notwendigen Bibliotheken
!pip install --upgrade gspread pandas

# Importieren der benötigten Bibliotheken
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet laden
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)
worksheet = sheet.worksheet('RM_KAU')

# Daten als Pandas DataFrame laden
data = pd.DataFrame(worksheet.get_all_records())

def check_duplicate_entries_and_highlight(data, cols_to_check):
    """
    Prüft Duplikate in den angegebenen Spalten und markiert diese in der Tabelle.

    Args:
        data (pd.DataFrame): Original-DataFrame.
        cols_to_check (list): Liste der zu prüfenden Spalten.

    Returns:
        pd.DataFrame: Fehler-DataFrame mit Zeilennummer, UserID, Spalte und Duplikaten.
    """
    errors = []

    # Daten in ein Array laden, um Anfragen an das Sheet zu minimieren
    data_array = data.to_numpy()
    col_indices = [data.columns.get_loc(col) for col in cols_to_check]
    user_id_index = data.columns.get_loc('UserID')  # Anpassung an die richtige Spalte

    for row_idx, row in enumerate(data_array):
        for col_idx in col_indices:
            cell_value = row[col_idx]
            if pd.notna(cell_value):
                entries = str(cell_value).split(',')
                duplicates = [x for x in set(entries) if entries.count(x) > 1]
                if duplicates:
                    # Fehler sammeln
                    errors.append({
                        'Row Number': row_idx + 2,  # Zeilennummer in Google Sheets
                        'UserID': row[user_id_index],
                        'Column': data.columns[col_idx],
                        'Duplicate Values': ', '.join(duplicates)
                    })
                    # Zellwert rot markieren
                    data_array[row_idx, col_idx] = f"#RED# {cell_value}"

    # Fehler-DataFrame erstellen
    error_df = pd.DataFrame(errors)

    # Daten zurück in DataFrame konvertieren
    updated_data = pd.DataFrame(data_array, columns=data.columns)

    return updated_data, error_df

# Zu prüfende Spalten definieren
cols_to_check = [col for col in data.columns if 'AB' <= col <= 'AH']

# Funktion anwenden
updated_data, error_df = check_duplicate_entries_and_highlight(data, cols_to_check)

# Fehler anzeigen
if not error_df.empty:
    print("Fehler gefunden:")
    print(error_df)
else:
    print("Keine Duplikate gefunden.")

# Aktualisierte Daten in das Google Sheet zurückschreiben
updated_values = updated_data.replace({"#RED# ": ""}, regex=True).values.tolist()
worksheet.update(updated_values)


###löscht alle felder mit komma alleine

In [ ]:
# prompt: Untersuche die Spalten "Maintenance KAU Roles, Analyse Reporting, Ersatzteilmanagem., Kalibrierung, EWM KAU, MDG EAM, MDG MM im frame: "data"
# schaue welche Felder mit einem "," beginnen statt mit einem "1201_"
# lösche dann das "," in dem gsheet selber

import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Drive und Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet öffnen (URL oder Sheet-ID einfügen)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)

# Tabellenreiter "RM_KAU" auswählen
worksheet = sheet.worksheet('RM_KAU')

# Daten als Pandas DataFrame laden
data = pd.DataFrame(worksheet.get_all_records())

columns_to_check = ["Maintenance KAU Roles", "Analyse Reporting", "Ersatzteilmanagem.", "Kalibrierung", "EWM KAU", "MDG EAM", "MDG MM"]

for column in columns_to_check:
  if column in data.columns:
    # Finde Zeilen, die mit einem ',' beginnen
    rows_to_fix = data[data[column].astype(str).str.startswith(',')]

    if not rows_to_fix.empty:
      print(f"Folgende Zeilen in Spalte '{column}' beginnen mit einem Komma und müssen korrigiert werden:")
      print(rows_to_fix[column])
      # Hier könnte man den Code zum Ändern der Werte in Google Sheets einfügen
      #  z.B. mit worksheet.update_cell(row_index, col_index, new_value)
      # ACHTUNG: Das direkte Ändern der Werte im DataFrame ändert nicht die Werte im Google Sheet!

      # Für die manuelle Korrektur in Google Sheets:
      print(f"\nBitte korrigiere die entsprechenden Zellen in Google Sheets (Spalte: '{column}')")

  else:
    print(f"Spalte '{column}' nicht gefunden.")

In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Drive und Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet öffnen (URL oder Sheet-ID einfügen)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)

# Tabellenreiter "RM_KAU" auswählen
worksheet = sheet.worksheet('RM_KAU')

# Daten als Pandas DataFrame laden
data = pd.DataFrame(worksheet.get_all_records())

columns_to_check = ["Maintenance KAU Roles", "Analyse Reporting", "Ersatzteilmanagem.", "Kalibrierung", "EWM KAU", "MDG EAM", "MDG MM"]

for column in columns_to_check:
  if column in data.columns:
    # Finde Zeilen, die mit einem ',' beginnen
    rows_to_fix = data[data[column].astype(str).str.startswith(',')]

    if not rows_to_fix.empty:
      print(f"Folgende Zeilen in Spalte '{column}' beginnen mit einem Komma und werden korrigiert:")

      # Iteriere durch die Zeilen, die korrigiert werden müssen
      for index, row in rows_to_fix.iterrows():
        # Ermitteln der Zellkoordinaten
        row_index = index + 2  # +2, da die Indexierung in Google Sheets bei 1 beginnt und die Kopfzeile eine Zeile einnimmt
        col_index = data.columns.get_loc(column) + 1  # +1, da die Spaltenindexierung in Google Sheets bei 1 beginnt

        # Den Wert in der Zelle aktualisieren (Komma entfernen)
        new_value = str(row[column]).lstrip(',')
        worksheet.update_cell(row_index, col_index, new_value)
        print(f"Zelle in Zeile {row_index}, Spalte {col_index} wurde aktualisiert: Von '{row[column]}' zu '{new_value}'")

  else:
    print(f"Spalte '{column}' nicht gefunden.")

###Kommatas löschen die zuviel

Optimierte Version mit Array - schneller


In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from gspread.utils import rowcol_to_a1 # Import rowcol_to_a1 function

# Authentifizierung für Google Drive und Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheet öffnen (URL oder Sheet-ID einfügen)
# Hier mit einem Backup File - "Copy of Role-Mapping KAU-Backup-23-12-24"
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Ied_PPiRmewmG_tj14j4h6iutl82LXpSmN5B-aJWb0I/edit?gid=2009282802#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)

# Tabellenreiter "RM_KAU" auswählen
worksheet = sheet.worksheet('RM_KAU')

# Daten als Pandas DataFrame laden
data = pd.DataFrame(worksheet.get_all_records())

columns_to_check = ["Maintenance KAU Roles", "Analyse Reporting", "Ersatzteilmanagem.", "Kalibrierung", "EWM KAU", "MDG EAM", "MDG MM"]

# Create a list to store cell updates
cell_updates = []

for column in columns_to_check:
    if column in data.columns:
        # Finde Zeilen, die mit einem ',' beginnen
        rows_to_fix = data[data[column].astype(str).str.startswith(',')]

        if not rows_to_fix.empty:
            print(f"Folgende Zeilen in Spalte '{column}' beginnen mit einem Komma und werden korrigiert:")

            # Iteriere durch die Zeilen, die korrigiert werden müssen
            for index, row in rows_to_fix.iterrows():
                # Ermitteln der Zellkoordinaten
                row_index = index + 2  # +2, da die Indexierung in Google Sheets bei 1 beginnt und die Kopfzeile eine Zeile einnimmt
                col_index = data.columns.get_loc(column) + 1  # +1, da die Spaltenindexierung in Google Sheets bei 1 beginnt

                # Den Wert in der Zelle aktualisieren (Komma entfernen)
                new_value = str(row[column]).lstrip(',')
                # Store the cell update as a tuple (cell coordinates, new value)
                cell_updates.append(((row_index, col_index), new_value))
                print(f"Zelle in Zeile {row_index}, Spalte {col_index} wurde aktualisiert: Von '{row[column]}' zu '{new_value}'")

    else:
        print(f"Spalte '{column}' nicht gefunden.")

# Update all cells in a single batch
if cell_updates:
    # Convert cell updates to a format suitable for worksheet.update()
    cell_range = [rowcol_to_a1(*coords) for coords, _ in cell_updates]
    new_values = [[value] for _, value in cell_updates]  # Values need to be in a 2D list

    # Update the cells
    worksheet.update(cell_range, new_values)

print(f"Die Spalten '{', '.join(columns_to_check)}' wurden aktualisiert.")

##Offenes

- wer heute eine 1080/1090 Rolle hat - Viewer egal von wo (Bsp.Compliance)
- Wer Techniker und Mechaniker ist braucht noch die MRO Rolle KAU und die L4L - checken wer ncoh nicht hat und eintragen
- Planer bekommt nur wer PK oder PM oder VG ist
